In [1]:
import numpy as np

$\newcommand{\diag}[1]{\text{diag}\left(#1\right)}$

# An extension to multiple occupations

Suppose that instead of separate sector level labor markets there are $\mathcal{O}$ occupations. Firms in each sector can use any occupation $o$ type labor in production. Our standard framework is the special case where there is exactly one occupation per sector and firms use only their own sector occupation as an input into production. 

## Simulating needed data

For now, I assume production functions are Cobb-Douglass, households have Cobb-Douglas preferences over final consumption goods, and the matching function is Cobb-Douglas. In principle, we don't need to be so restrictive about the functional forms of production and matching. Instead all of the relevant information about production technologies is captured by the levels and changes of the elasticities outline below. Any two production functions with the same elasticities and changes in elasticities would generate the same responses to shocks.

For each sector $i$, we need the elasticity of production of good $j$ with respect to the intermediate input from each other sector $j$, $\varepsilon^{f_i}_{x_{ij}}$, and the elasticity of production with respect to occupation $o$ labor input $N_{io}$, $\varepsilon^{f_i}_{N_{io}}$. I sample elasticites of production on the unit simplex using an exponential transformation of the normal distribution. To start, for each industry, draw
\begin{align*}
    \left(z_{i1}, \cdots,z_{iJ},z_{N_{i1}}, \cdots, z_{N_{i\mathcal{O}}}\right) \sim N\left(\mu,\Sigma\right)
\end{align*}
Then define
\begin{align*}
    \varepsilon^{f_i}_{x_{i1}} = \frac{e^{z_{i1}}}{\sum_{o=1}^{\mathcal{O}}e^{z_{N_{io}}}+\sum_{j=1}^{J}e^{z_{ij}}}
\end{align*}
And equivalently for all other sectors. We also need the elasticities of demand with respect to sector $i$'s output, $\varepsilon^{\mathcal{D}}_{c_i}$. Notice that we can treat the final demand as another production sector that uses no labor input. We can sample the demand elasticities using the same procedure outlined above.

Finally, I sample the matching elasticity with respect to vacancies from a uniform distribution on $[0,1]$. 

We let $\Omega$ be the matrix of input elasticities for each sector and $\Psi = (\bm{I}-\bm{\Omega})^{-1}$ be the Leontif inverse.

In [2]:
# Size of network
J = 4 # number of sectors
O = 2 # number of occupations

# Sampling input-output matrix entries 
μ = np.zeros(J+O)
Σ = np.eye(J+O)
z_draws = np.random.multivariate_normal(μ, Σ, J+1)
elasticity_draws = np.exp(z_draws)/np.sum(np.exp(z_draws),1).reshape((J+1,1))

# Elasticities
epsD = elasticity_draws[0,:-O]/np.sum(elasticity_draws[0,:-O])
epsD = epsD.reshape(J,1)
Omega = elasticity_draws[1:,:-O]
Psi = np.linalg.inv(np.eye(J)-Omega)
epsN = elasticity_draws[1:,J:]

# Drawing elasticity of matching function wrt to U
ν = np.random.uniform(size=O)
epsQ =  -ν.reshape((O,1))
epsF = np.ones((O,1)) + epsQ
epsQ = epsQ.reshape((O,1))
epsF = epsF.reshape((O,1))
curlyQ = np.diag(epsQ.flatten())
curlyF =  np.diag(epsF.flatten())

np.sum(Omega,1) + np.sum(epsN,1) # checking constant returns holds

array([1., 1., 1., 1.])

We need recruiter producer ratios $\tau_o(\theta_o)$ in each occupational labor market. Landais, Michaillat, and Saez (2018) find that the share of recruiters in the US workforce averages around 2.3 percent. I sample recruiter producer ratios uniformly on the interval $[0,0.046]$ to roughly match this fact.

In [3]:
tau = np.random.uniform(low=0,high=0.046,size=(O,1))
curlyT = np.diag(tau.flatten())
curlyT

array([[0.01588763, 0.        ],
       [0.        , 0.02885142]])

Finally, in addition to the data we need assuming one occupation per sector, we need data on employment by sector-occupation pair at the initial equilibrium. For each $i$ and $o$ we need
\begin{align*}
    l_{io} = \frac{\varepsilon^{f_i}_{N_{io}}p_i y_i}{w_o}
\end{align*}
For the purposes of this simple example, we assume that all prices, wages, and quantities are the same and normalized to $1$. This implies that $l_{io} = \varepsilon^{f_i}_{N_{io}}$. 

In [4]:
# labor usage at initial equilibrium
l = epsN
curlyL = l.T/np.sum(l.T,1).reshape((O,1)) 

## Defining shocks
We are interested in the response of sector level and aggregate output and employment to technology shocks $d\log\bm{A}$ and labor force shocks $d\log \bm{H}$. The code below defines the shocks we feed into the model.

In [5]:
# Technology shocks
dlog_A = 0.01*np.ones((J,1))
dlog_H = 0.01*np.zeros((O,1))

## Defining how wages adjust
Since there are mutual gains from trade once an unemployed worker and a firm meet, wages are not pinned down uniquely in models featuring search and matching frictions in the labor market. We must therefore impose a wage schedule, an assumption about how wages change in response to fundamentals, in order to close the model. 

Let $w_o$ be the nominal occupation $o$ wage. It turns out to be convenient to write everything in terms of wages net of occupation share weighted sectoral prices $d\log \bm{w} - \bm{\mathcal{L}} d\log \bm{p}$, where 
\begin{align*}
    \bm{\mathcal{L}} = \begin{bmatrix}
        \frac{l_{11}}{L^d_1} & \frac{l_{21}}{L^d_{1}} & \cdots & \frac{l_{J1}}{L^d_{1}} \\
        \frac{l_{12}}{L^d_2} & \frac{l_{22}}{L^d_2} & \cdots  & \frac{l_{J2}}{L^d_2} \\
         \vdots & \vdots & \ddots & \vdots \\
        \frac{l_{1\mathcal{O}}}{L^d_{\mathcal{O}}} & \frac{l_{2\mathcal{O}}}{L^d_{\mathcal{O}}} & \cdots & \frac{l_{J\mathcal{O}}}{L^d_{\mathcal{O}}}
    \end{bmatrix}
\end{align*}

We need to specify the elasticity of adjusted wages to technology shocks and labor force shocks in each sector, $\left\{\left\{\varepsilon^{w_o}_{A_{oi}}\right\}_{i=1}^{J},\left\{\varepsilon^{w_o}_{H_{ok}}\right\}_{k=1}^{\mathcal{O}}\right\}_{o=1}^{\mathcal{O}}$. For instance, a simple assumption is that the response of wages to productivity shocks is proportional to $\varepsilon^{f_i}_{N_{io}}$ and how adjusted wages respond to labor force changes is proportional to $\varepsilon^{Q_o}_{\theta_o}$. In general, we express changes in wages as a function of changes in productivity and the labor force. 
\begin{align}
    d\log \bm{w} - \bm{\mathcal{L}}d\log\bm{p}&= \bm{\Lambda_{A}} d\log \bm{A} + \bm{\Lambda_{H}} d\log \bm{H} \tag{1}
\end{align}
Where $\bm{\Lambda_{A}}$ contains wage elasticities to productivity changes and $\bm{\Lambda_{H}}$ contains wage elasticities to labor force changes. We consider the class of wage schedules such that 
\begin{align*}
    d\log\bm{w} - \bm{\mathcal{L}} d\log\bm{p} &= \gamma_A\bm{\mathcal{L}} \bm{\Psi} d\log A + \gamma_H \left[\bm{\mathcal{L}}\bm{\Psi}\bm{\varepsilon^f_N}-\bm{I}\right] d\log\bm{H}
\end{align*}
Hulten's theorem holds as special case of this assumption where production and preferences are Cobb-Douglas, $\gamma_A = 1$, and $d\log\bm{H} = 0$. Therefore, this assumption allows us to explore the quantitative implications of deviations from the Hulten knife edge case by varying $\gamma_A$ and $\gamma_H$. Clearly, even allowing for general $\gamma_A$ and $\gamma_H$, this is a very restrictive assumption about how wages change. We view it as a convenient expositional tool.  

In [40]:
def WageElasticityFunc(gamma_A, gamma_H, Psi, curlyL, epsN):
    epsW_A = gamma_A * curlyL @ Psi
    epsW_H = gamma_H * (curlyL @ Psi @ epsN - np.eye(curlyL.shape[0]))
    return epsW_A, epsW_H

In [45]:
# Wage elasticities
epsW_A, epsW_H = WageElasticityFunc(1, 1, Psi, curlyL, epsN)
epsW_A

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [7]:
# Calculating the log change in wages
def WageFunc(dlog_A, dlog_H, epsW_A, epsW_H):
    dlog_wR = epsW_A @ dlog_A + epsW_H @ dlog_H
    return dlog_wR

In [8]:
#How wages change
dlog_wR = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
dlog_wR

array([[0.04989758],
       [0.03758793]])


## Tightness propagation
We with wage changes in hand, we solve for first order changes in tightness in terms of $d\log\bm{A}$, $d\log\bm{H}$, and $d\log\bm{w}$. The general formula for changes in tightness is
\begin{align}
    d\log\bm{\theta} &= \left[\bm{\mathcal{F}}-\bm{\Xi_{\theta}}\right]^{-1}\left[\bm{\mathcal{L}}\bm{\Psi}d\log\bm{A} - \left[d\log\bm{w}-\bm{\mathcal{L}} d\log\bm{p}\right] + \left[\bm{\mathcal{L}\Psi \varepsilon^{f}_{N}}-\bm{I}\right]d\log\bm{H}\right] \nonumber\\
    &+ \left[\bm{\mathcal{F}}-\bm{\Xi_{\theta}}\right]^{-1} \left[\diag{\bm{\mathcal{L}}d\log\bm{\varepsilon^f_N}} +\bm{\mathcal{L}}\bm{\Psi}\left[\left(\diag{\bm{\Omega}\bm{1}} - \bm{\Omega}\right)d\log \bm{\lambda}-d\log \bm{\mathcal{E}} \right] \right] \tag{2}
\end{align}
Where $\bm{\Xi_{\theta}} = \bm{\mathcal{L}}\bm{\Psi}\bm{\varepsilon^{f}_N}\left[\bm{\mathcal{F}}+\bm{\mathcal{Q\Tau}}\right]$ and $d\log\bm{\mathcal{E}} = \diag{\bm{\varepsilon^f_N}d\log{\bm{\varepsilon^f_N}}'}$. Notice, all terms in the second line are zero assuming Cobb-Douglas production technology. 

In [9]:
def curlyEFunc(dlog_epsN,epsN):
    J = epsN.shape[0]
    curlyE = np.diag(epsN @ dlog_epsN.T).reshape(J,1)
    return curlyE

In [10]:
def ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN):
    O = dlog_H.shape[0]
    
    # Creating matrices

    Xi = curlyL @ Psi @ epsN @ (curlyF + curlyQ @ curlyT)
    inv_mat = np.linalg.inv(curlyF - Xi)
    I = np.eye(O)
    
    # Contribution of different components
    Cw = -inv_mat 
    Ca = inv_mat @ curlyL @ Psi
    Ch = inv_mat @ (curlyL @ Psi @ epsN - I)
    Ce = -inv_mat @ curlyL @ Psi
    Cλ = inv_mat @ curlyL @ Psi @ (np.diag(np.sum(Omega,1)) - Omega)

    # Change in tightness
    dlog_theta = Cw @ dlog_wR + Ch @ dlog_H + inv_mat @ np.diag(curlyL @ dlog_epsN).reshape((O,1)) + Cλ @ dlog_lam + Ce @ curlyE + Ca @ dlog_A
    
    return dlog_theta

Assuming Cobb-Douglas production implies $d\log\bm{\varepsilon^f_N} = d\log \bm{\lambda} = 0$.

In [11]:
dlog_epsN = np.zeros((J,O))
curlyE = curlyEFunc(dlog_epsN,epsN)
curlyE

array([[0.],
       [0.],
       [0.],
       [0.]])

In [12]:
dlog_lam = np.zeros_like(dlog_A)
dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN)
dlog_theta 

array([[-0.04012872],
       [-0.16726611]])

## Price and output propagation
With changes in tightness in hand, we can now work out how prices and sectoral production changes in response to technology and labor force shocks. Price changes are given by 
\begin{align}
           \left(\bm{I} - \bm{\Psi \varepsilon^{f}_N\mathcal{L}}\right)d\log\bm{p} &= \bm{\Psi}\left[\bm{\varepsilon^{f}_{N}}\left[d\log\bm{w}-\bm{\mathcal{L}}d\log\bm{p}-\bm{\mathcal{Q}\mathcal{\Tau}}d\log\bm{\theta}\right]-d\log\bm{A}\right] \tag{3}
\end{align}
In general, in order to identify price changes, we will need to select a numeraire since the matrix $\left(\bm{I} - \bm{\Psi \varepsilon^{f}_N\mathcal{L}}\right)$ is not invertible. 

In [14]:
def PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, curlyT, curlyL, num = 0):
    # Contributions of different components
    Cw = Psi @ epsN
    Ctheta = -Cw @ curlyQ @ curlyT
    Ca = -Psi
    # Imposing numeraire
    Cw[num, :] = 0
    Ctheta[num, :] = 0
    Ca[num, :] = 0
    # Inverse matrix
    Xi = np.eye(Psi.shape[0]) - Psi @ epsN @ curlyL
    Xi[num, :] = 0
    Xi[num, num] = 1
    inv_mat = np.linalg.inv(Xi)

    # Price changes
    dlog_p = inv_mat @ (Cw @ dlog_wR + Ctheta @ dlog_theta + Ca @ dlog_A)
    return dlog_p


In [18]:
dlog_p = PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, curlyT, curlyL, num=0)
dlog_p

array([[ 0.        ],
       [ 0.00310563],
       [ 0.00442684],
       [-0.00622718]])

And output changes are given by
\begin{align}
       d\log \bm{y} &= \bm{\Psi}\left(d\log\bm{A} + \bm{\varepsilon^{f}_{N}}\left(\bm{\mathcal{F}}+\bm{\mathcal{Q}} \bm{\mathcal{\Tau}}\right)d\log \bm{\theta} + \bm{\varepsilon^{f}_{N}} d\log\bm{H}\right)\nonumber \\
    &-\bm{\Psi}d\log \bm{\mathcal{E}} + \bm{\Psi}\left(\diag{\bm{\Omega}\bm{1}} - \bm{\Omega}\right)d\log \bm{\lambda} \tag{4}
\end{align}


In [19]:
def OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE):
    # Contributions of different coponents
    Ca = Psi
    Ch = Psi @ epsN
    Ctheta = Ch @ (curlyF + curlyQ @ curlyT)
    Ce = -Psi
    Cλ = Psi @ (np.diag(np.sum(Omega,1)) - Omega)
    
    # Output changes
    dlog_y = Ca @ dlog_A + Ctheta @ dlog_theta + Ch @ dlog_H + Ce @ curlyE + Cλ @ dlog_lam

    return dlog_y

In [20]:
dlog_y = OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.01332981],
       [0.01022418],
       [0.00890297],
       [0.01955699]])

With Cobb-Douglas production, real output should change by the same amount in each sector. This provides an overall check of our code given the Cobb-Douglas assumption.

In [21]:
# Change in nominal GDP
dlog_p + dlog_y

array([[0.01332981],
       [0.01332981],
       [0.01332981],
       [0.01332981]])

We can now check consistency. We should get the same change in labor by either calculating the change in labor supply
\begin{align}
    d\log \bm{L^s} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}
Or the change in labor demand
\begin{align}
d\log \bm{L^d} = \bm{\mathcal{L}} d\log\bm{y} + \diag{\bm{\mathcal{L}}d\log \bm{\varepsilon^f_N}} - \left[d\log\bm{w} -\bm{\mathcal{L}} d\log\bm{p}\right] \tag{6}
\end{align}

In [25]:
def LaborSupply(dlog_H,dlog_theta,curlyF):
    dlog_Ls = curlyF @ dlog_theta + dlog_H
    return dlog_Ls

def LaborDemand(dlog_wR, dlog_y, dlog_p, dlog_epsN, curlyL):
    O = dlog_wR.shape[0]
    dlog_Ld = curlyL @ dlog_y + np.diag(curlyL @ dlog_epsN).reshape((O,1)) - dlog_wR
    return dlog_Ld

Consistency requires
\begin{align*}
    d\log \bm{L^d} - d\log\bm{L^s} = 0
\end{align*}

In [26]:
LaborDemand(dlog_wR, dlog_y, dlog_p, dlog_epsN, curlyL) - LaborSupply(dlog_H,dlog_theta,curlyF)

array([[-6.9388939e-18],
       [ 6.9388939e-18]])

The aggregate response of output to technology and labor supply shocks is given by 
\begin{align}
   d\log Y = \bm{\varepsilon^{\mathcal{D}}_{c}}' \left[ d\log \bm{\varepsilon^{\mathcal{D}}_{c}}+d\log\bm{y}-d\log\bm{\lambda}\right] \tag{7}
\end{align}

In [79]:
def AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD):
    dlog_aggY = epsD.T @ (dlog_epsD + dlog_y - dlog_lam)
    return dlog_aggY

Assuming Cobb-Douglas, $d\log\bm{\varepsilon^{\mathcal{D}}_{c}}=0$. 

In [81]:
dlog_epsD = np.zeros_like(dlog_y)
dlog_aggY = AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([[4.13797772]])

## Special case with one occupation per sector
Basic setup.

In [75]:
# Size of network
J = 4 # number of sectors
O = J # number of occupations

# Sampling input-output matrix entries 
μ = np.zeros(J+O)
Σ = np.eye(J+O)
z_draws = np.random.multivariate_normal(μ, Σ, J+1)
elasticity_draws = np.exp(z_draws)/np.sum(np.exp(z_draws),1).reshape((J+1,1))

# Elasticities
epsD = elasticity_draws[0,:-O]/np.sum(elasticity_draws[0,:-O])
epsD = epsD.reshape(J,1)
Omega = elasticity_draws[1:,:-O]
Psi = np.linalg.inv(np.eye(J)-Omega)
epsN = np.diag(np.sum(elasticity_draws[1:,J:],1))
l = epsN

# Drawing elasticity of matching function wrt to U
ν = np.random.uniform(size=O)
epsQ =  -ν.reshape((O,1))
epsF = np.ones((O,1)) + epsQ
epsQ = epsQ.reshape((O,1))
epsF = epsF.reshape((O,1))
curlyQ = np.diag(epsQ.flatten())
curlyF =  np.diag(epsF.flatten())

np.sum(Omega,1) + np.sum(epsN,1) # checking constant returns holds

array([1., 1., 1., 1.])

In [76]:
tau = np.random.uniform(low=0,high=0.046,size=(O,1))
curlyT = np.diag(tau.flatten())
curlyT

array([[0.02319921, 0.        , 0.        , 0.        ],
       [0.        , 0.00769664, 0.        , 0.        ],
       [0.        , 0.        , 0.0038407 , 0.        ],
       [0.        , 0.        , 0.        , 0.0154689 ]])

Wage changes.

In [77]:
# Wage elasticities
epsW_A = np.random.uniform(low=0,high=2,size=(O,J))
epsW_H = np.random.uniform(low=-2,high=0,size=(O,O))

# Technology shocks
dlog_A = 0.01*np.ones((J,1))
dlog_H = 0.01*np.zeros((O,1))

dlog_w = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
dlog_w = np.zeros_like(dlog_w)
dlog_w

array([[0.],
       [0.],
       [0.],
       [0.]])

Changes in tightness.

In [78]:

dlog_epsN = np.zeros((J,O))
curlyE = curlyEFunc(dlog_epsN,epsN)
dlog_lam = np.zeros_like(dlog_A)

dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN, num=0)
dlog_theta

TypeError: ThetaFunc() missing 1 required positional argument: 'epsN'

Changes in prices.

In [ ]:
dlog_p = PriceFunc(dlog_A, dlog_w, dlog_theta, Psi, curlyQ, epsN, curlyT)
dlog_p

array([[-1.70002901e-16],
       [-1.60978718e-02],
       [-2.21109558e-02],
       [-8.86068649e-03]])

Changes in Output.

In [ ]:
dlog_y = OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_lam, Psi, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.04055621],
       [0.05665408],
       [0.06266716],
       [0.04941689]])

Checking results.

In [ ]:
dlog_p + dlog_y

array([[0.04055621],
       [0.04055621],
       [0.04055621],
       [0.04055621]])

In [ ]:
LaborDemand(dlog_w, dlog_y, dlog_p, dlog_epsN, l) - LaborSupply(dlog_H,dlog_theta,curlyF)

array([[-1.80411242e-16],
       [ 0.00000000e+00],
       [ 1.38777878e-17],
       [ 0.00000000e+00]])